In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

C:\Users\LASYA\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
model = load_model('emojinator.h5')

W0816 15:46:35.110440  4468 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 15:46:36.343177  4468 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0816 15:46:36.774208  4468 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0816 15:46:36.896070  4468 deprecation_wrapper.py:119] From C:\Users\LASYA\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 15:46:36.898064  4468 deprecation_wrapper.py:119] From C:\Users\LASYA\An

In [3]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

In [6]:
def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [7]:
keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
3 0.65795153
1 0.9941736
1 0.9997905
1 0.9583697
1 0.84037745
11 0.9747926
1 0.91978353
1 0.99856645
1 0.9920831
1 0.99549544
1 0.9990934
1 0.9975248
11 0.51650596
1 0.9996092
1 0.9966737
1 0.99542195
1 0.50999725
1 0.99764436
11 0.9317811
11 0.9855871
11 0.6938831
1 0.94401234
11 0.906222
1 0.5484984
11 0.9626798
11 0.89393944
1 0.81749576
11 0.9194037
11 0.839661
1 0.99993634
1 0.8036366
1 0.9990526
1 0.9996823
1 0.84698904
11 0.5205206
11 0.87822497
1 0.73628885
11 0.8409708
11 0.9655758
1 0.8318825
11 0.88453054
1 0.6532864
11 0.90248823
1 0.70556897
1 0.9272844
11 0.78899664
11 0.87924314
1 0.57586163
1 0.5039057
11 0.5843904
1 0.7935247
1 0.7252934
1 0.70301
11 0.9334494
1 0.763073
11 0.97903836
11 0.9338825
1 0.9129172
1 0.8087203
1 0.73196316
1 0.6726761
11 0.8849593
11 0.98292947
11 0.8097043
1 0.9838378
11 0.9707506
11 0.99036586
11 0.9901529
11 0.93096036
1 0.9893895
1 0.49768198
11 0.9818672
11 0.9956189
1 0.9399816
9 0.62061864
5 0.55616283
9 0

3 0.6073231
11 0.6597304
11 0.742448
11 0.58940244
11 0.70865035
11 0.65799475
2 0.4709723
5 0.9964903
11 0.43401513
5 0.89539135
5 0.99874425
5 0.996896
5 0.8776474
5 0.9949922
5 0.99202055
5 0.99414355
5 0.87838036
5 0.49121505
5 0.99953973
5 0.99962497
5 0.585274
5 0.9076552
5 0.9952655
5 0.95071507
5 0.97848856
5 0.99923813
5 0.9972434
5 0.82784164
5 0.59240705
5 0.9945839
7 0.41592667
5 0.94212556
5 0.62938935
5 0.8661355
5 0.9216476
5 0.9649697
5 0.9668701
5 0.5545578
5 0.94033444
5 0.99930394
5 0.946959
5 0.9979467
11 0.7579887
11 0.6074155
11 0.63156605
11 0.7785477
11 0.93075645
11 0.7037647
11 0.8778184
11 0.97294664
11 0.94918334
11 0.99683565
11 0.9940339
11 0.98339105
11 0.99686146
11 0.9971724
11 0.94722044
11 0.9996062
11 0.9992418
11 0.95476496
11 0.97810763
11 0.78839797
11 0.99967766
11 0.9766813
11 0.966867
11 0.99820805
11 0.9995524
11 0.98822767
3 0.873389
11 0.88488865
11 0.8124306
11 0.83055913
11 0.80193466
11 0.9801476
11 0.9721552
11 0.9115566
11 0.9799502
11 

2 0.4141984
4 0.30132377
6 0.2615765
5 0.51893044
5 0.38271788
6 0.41429254
6 0.702549
7 0.290202
5 0.49360478
6 0.2678638
5 0.28821486
6 0.8365011
4 0.26704878
5 0.45526272
7 0.65770817
7 0.5717595
6 0.6300662
6 0.4332096
3 0.4112937
6 0.24609585
3 0.18470517
5 0.40189585
5 0.40707242
5 0.4880079
6 0.54346836
6 0.39327338
5 0.30870795
5 0.38453925
7 0.3306475
8 0.2405873
5 0.3734668
7 0.40577698
7 0.7802067
4 0.43898866
3 0.478315
5 0.44577336
3 0.2329925
6 0.3381175
6 0.6280178
6 0.9385235
5 0.6087754
6 0.25907978
7 0.26569355
8 0.5435942
7 0.3572405
7 0.4414475
6 0.43748468
4 0.26641527
5 0.49195212
7 0.25876713
6 0.58947325
6 0.3317773
6 0.45052472
7 0.45137385
5 0.4078215
5 0.45683485
7 0.3711089
5 0.74759865
5 0.6459772
5 0.2520135
4 0.9284789
5 0.70125425
4 0.24837358
6 0.48949683
5 0.33254513
5 0.69692975
5 0.39702132
6 0.2264429
4 0.3346763
5 0.606959
5 0.3420941
6 0.7296051
5 0.6957086
5 0.6133431
5 0.39272
7 0.34411
7 0.43784675
7 0.6040696
7 0.49078402
6 0.4340499
5 0.18346

10 0.97837836
10 0.6543282
4 0.9345399
10 0.9953987
4 0.5610856
4 0.5037559
10 0.89288557
10 0.6733132
10 0.7475687
10 0.9924992
10 0.9976253
10 0.99499303
4 0.6140997
6 0.8074672
10 0.9969439
10 0.9892966
10 0.93992174
4 0.85449505
10 0.86436206
10 0.5875588
11 0.9569993
10 0.9670748
4 0.5224565
6 0.90578324
10 0.9932973
10 0.836345
10 0.7672104
10 0.90499836
10 0.99185
6 0.658002
6 0.5253391
10 0.99767715
10 0.9601781
10 0.9952868
10 0.94444025
10 0.99756396
6 0.6619181
10 0.9774566
10 0.9529495
10 0.9997708
6 0.42844817
10 0.9988589
4 0.38951638
3 0.632666
10 0.99995065
4 0.6075925
10 0.9880189
6 0.94327396
10 0.98159456
10 0.98803747
10 0.6518299
10 0.86939466
6 0.9702417
10 0.48841375
7 0.54698545
10 0.94733757
10 0.9992029
10 0.96467364
10 0.84703827
6 0.95898825
10 0.99719465
4 0.69536567
10 0.99284345
4 0.8860315
10 0.94064075
10 0.86798203
3 0.7320598
3 0.49916983
10 0.9768322
10 0.9100972
10 0.60268956
10 0.9998652
10 0.67768556
6 0.51078314
6 0.95326316
6 0.7382773
6 0.94386

10 0.3659008
10 0.38009182
5 0.91400677
6 0.7570686
10 0.9818927
6 0.86694336
10 0.8826635
6 0.5570014
10 0.9324419
9 0.93675476
9 0.9265267
6 0.5386308
1 0.860934
1 0.93477654
1 0.85703695
1 0.75856227
1 0.84034866
1 0.74232614
1 0.97834194
1 0.9999981
1 0.9922712
1 0.9394144
1 0.5824569
1 0.9561067
1 0.76867163
1 0.87202394
7 0.8634066
1 0.97503304
1 0.96401834
11 0.3554447
5 0.52409154
1 0.9786728
1 0.522555
11 0.83529603
1 0.54264164
1 0.9229272
1 0.99992085
7 0.42645714
1 0.6333023
5 0.7363684
9 0.42782858
1 0.80173117
1 0.9893176
11 0.9160233
6 0.36245197
10 0.8771122
1 0.99992204
1 0.5867476
1 0.95289326
1 0.5472287
7 0.42182404
1 0.7770296
11 0.20909339
9 0.9319027
11 0.47857118
7 0.4616981
11 0.5664756
11 0.28162745
7 0.3294373
9 0.38588485
9 0.8996199
11 0.78788805
11 0.8678709
9 0.528562
9 0.41739705
11 0.77336544
11 0.9905625
11 0.9209369
9 0.538588
10 0.9868831
11 0.88787615
10 0.9592077
1 0.9972475
1 0.8022447
9 0.85917234
9 0.32954156
1 0.9667659
11 0.8697864
1 0.6090809

...

5 0.99958247
5 0.961963
5 0.998995
5 0.9999969
5 0.99996233
1 0.99277025
5 0.99999976
5 1.0
5 0.99999547
5 0.9999993
5 0.9999167
5 0.99998176
5 0.9999993
5 0.9999976
5 0.9893784
5 0.9526611
5 0.728476
4 0.71137744
5 0.9999988
5 0.9999883
5 0.9999949
5 0.993244
11 0.78959936
11 0.6904644
5 0.9999995
4 0.6134325
5 0.99997187
4 0.5168277
5 1.0
5 0.9906262
5 0.9996737
2 0.45498955
1 0.9866072
10 0.28696007
2 0.37712225
2 0.99717486
2 0.51289773
2 0.77291495
2 0.50692767
1 0.74988484
2 0.62394357
5 0.99998486
8 0.9987153
7 0.69886297
2 0.89355206
3 0.91104966
2 0.6471513
2 0.8161139
1 0.92219627
8 0.89942724
4 0.89568734
4 0.87390256
5 0.9979565
5 0.99997413
5 0.9996269
5 0.9990627
10 0.99437547
5 0.993005
5 0.9997447
5 0.9999827
5 0.9990829
5 0.99999297
5 0.9996087
5 0.9999012
5 0.99994624
5 0.99963427
5 0.99999654
5 0.99996877
5 0.99996006
5 0.99999714
5 0.9997404
5 0.9999789
5 0.9999933
5 0.9999956
5 0.9999871
5 0.9999832
5 0.9999827
5 0.99998987
5 0.9996018
4 0.9594584
5 0.9951698
7 0.3

3 0.9764256
3 0.92148125
3 0.9312913
3 0.72762406
3 0.9788029
3 0.9030275
3 0.95830566
3 0.7204337
4 0.70926625
4 0.89664
3 0.640245
4 0.823549
4 0.80445755
4 0.98060226
4 0.84054005
4 0.9777851
4 0.9627164
4 0.85054016
4 0.93954444
3 0.7604228
4 0.6458751
4 0.7528994
4 0.99118596
4 0.7551665
4 0.6799491
3 0.88531625
4 0.8818704
4 0.7287943
3 0.6670673
3 0.55269873
4 0.887504
3 0.5182919
3 0.6048458
4 0.9327907
4 0.84716326
4 0.865835
4 0.8065569
4 0.9092883
7 0.6710925
4 0.92443115
4 0.9554011
4 0.8617433
3 0.59234035
4 0.8795339
4 0.8081892
7 0.6038356
4 0.54197973
4 0.494229
7 0.7107958
7 0.68417156
4 0.97985667
4 0.71452326
4 0.7499817
3 0.61540025
3 0.86637145
3 0.7466788
3 0.9039908
3 0.90248317
3 0.8470184
3 0.8370901
3 0.86940867
3 0.8195618
3 0.62902045
3 0.91863006
3 0.8545112
3 0.927715
3 0.69855887
3 0.8170719
3 0.78135604
3 0.9080867
3 0.87069976
3 0.801685
3 0.95901346
3 0.6178391
3 0.7663352
3 0.6991689
3 0.68451667
3 0.814696
3 0.916646
3 0.8691797
3 0.8595557
3 0.94978

6 0.9856228
6 0.94705737
6 0.98342943
6 0.98795915
6 0.9880679
6 0.97422224
6 0.9116894
6 0.94527495
6 0.7487002
6 0.83366144
6 0.96594685
6 0.9746474
6 0.9170677
6 0.94425625
4 0.8720137
7 0.28439116
5 0.81204176
5 0.9633542
5 0.9997404
5 0.8814412
9 0.56404775
5 0.9125606
5 0.96382403
5 0.9622763
9 0.88313234
5 0.7081472
4 0.9439927
4 0.77782017
9 0.38568833
9 0.4252696
5 0.9933743
5 0.39859125
7 0.8300482
5 0.99984884
5 0.9999651
5 0.9999833
5 0.99999344
5 0.9916899
4 0.4922818
8 0.73076653
10 0.988236
7 0.71730655
3 0.9805046
3 0.6561103
3 0.47398823
1 0.9322425
6 0.65567654
6 0.89890814
6 0.6196832
10 0.97306156
10 0.99636275
10 0.97453415
10 0.99362856
10 0.9443549
10 0.9423531
10 0.97370726
10 0.5801715
10 0.8032537
10 0.84307176
10 0.8121492
10 0.70684904
10 0.5066818
6 0.44781566
10 0.36843908
6 0.60377383
6 0.7359804
6 0.9462659
6 0.40393412
6 0.7162029
6 0.657812
6 0.55679256
6 0.7936748
6 0.79823536
6 0.87972933
6 0.97866994
6 0.98010457
6 0.9531413
6 0.7775802
6 0.7694577


8 0.9646164
1 0.8129621
8 0.8752323
1 0.897556
8 0.90364313
8 0.82102555
1 0.80735123
1 0.6364869
8 0.70832825
8 0.94817245
1 0.95900035
1 0.972029
8 0.8600699
1 0.9665196
1 0.96716726
8 0.93443865
8 0.86280936
1 0.6322229
8 0.92837656
8 0.8299211
8 0.86089975
8 0.8699717
8 0.9481594
8 0.96424407
8 0.87110204
8 0.96393114
8 0.8538342
1 0.9491116
8 0.959077
8 0.94423836
8 0.96480757
8 0.97692704
8 0.9776072
8 0.9823632
8 0.95984846
8 0.97719246
8 0.9805567
1 0.67929137
8 0.72736746
1 0.96845925
1 0.9541083
1 0.99524575
1 0.9975617
1 0.99683374
1 0.99887174
1 0.9965823
1 0.9980107
1 0.9881493
1 0.9990421
1 0.9990355
1 0.99932575
1 0.9994104
1 0.9998647
1 0.9997471
1 0.99983454
1 0.99980944
1 0.9998759
1 0.9999107
1 0.9996774
1 0.99948657
1 0.9947241
1 0.992031
1 0.9984041
9 0.57309395
1 0.97486997
1 0.999315
1 0.9990339
1 0.9997389
1 0.99883515
1 0.99864787
1 0.9994087
1 0.99986625
1 0.9929931
1 0.9972632
1 0.9951757
1 0.92381227
1 0.7169533
1 0.9987577
1 0.99910814
1 0.99946004
1 0.9997

KeyboardInterrupt: 